<div style="text-align:center; margin:auto">
<h1>Ischemic Stroke Visualization Dashboard</h1>
</div>

<div style="text-align:center; margin:auto">
<img src='./brainclot.jpg' height=400px width=600px>
</div>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact

In [12]:
df = pd.read_csv('stroke.csv')

In [3]:
discharge_time = df['Discharge Time']
discharge_time = pd.to_datetime(discharge_time)

admission_time = df['Admission Time']
admission_time = pd.to_datetime(admission_time)

df['Admit Duration'] = discharge_time - admission_time

In [4]:
emergency_time = df['Emergency Dept Time']
emergency_time = pd.to_datetime(emergency_time)

df['Emergency Response Time'] = admission_time - emergency_time

In [5]:
df_com = df[df['Comorbidities']==True]
df_com_not = df[df['Comorbidities']==False]

## Demographics and Analysis with Comorbodities

In [11]:
@interact(Factor=['Age','Gender','Emergency Response Time', 'Admit Duration'], Show_Comorbidities=True)
def plot_demographic(Factor, Show_Comorbidities):

    fig, ax = plt.subplots(figsize=(10,6))
    if Show_Comorbidities:
        if not (Factor == 'Emergency Response Time' or Factor == 'Admit Duration'):
            sns.histplot(df_com[Factor], label=f'{Factor} Comorbidities', element='step', stat='frequency') 
            sns.histplot(df_com_not[Factor], label=f'{Factor} Non-Comorbidities', element='step', stat='frequency') 
        else:
            if Factor == 'Admit Duration':
                sns.histplot(df_com[Factor].dt.total_seconds() / 86400, label=f'{Factor} Comorbidities', element='step', stat='frequency')
                sns.histplot(df_com_not[Factor].dt.total_seconds() / 86400, label=f'{Factor} Non-Comorbidities', element='step', stat='frequency')
            else:
                sns.histplot(df_com[Factor].dt.total_seconds() / 3600, label=f'{Factor} Comorbidities', element='step', stat='frequency')
                sns.histplot(df_com_not[Factor].dt.total_seconds() / 3600, label=f'{Factor} Non-Comorbidities', element='step', stat='frequency')
    else:
        if not (Factor == 'Emergency Response Time' or Factor == 'Admit Duration'):
            sns.histplot(df[Factor], label=Factor, element='step') 
        else:
            if Factor == 'Admit Duration':
                sns.histplot(df[Factor].dt.total_seconds() / 86400, label=Factor, element='step')
            else:
                sns.histplot(df[Factor].dt.total_seconds() / 3600, label=Factor, element='step')
        
    ax.legend(loc='upper left')
    plt.show()
    

interactive(children=(Dropdown(description='Factor', options=('Age', 'Gender', 'Emergency Response Time', 'Adm…